LLMs的直接输出是文本，没用结构性，导致LLMs后续的操作存在不确定性，无法构建后续应用。一个比较常见的例子是：从特定文本提取某些字段的数据保存到数据库中。

以下我们问模型一个问题，比较没经过结构化、结构化后的输出

In [5]:
from langchain_ollama import ChatOllama
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:7b"
llm  = ChatOllama(model=model_name)

response=llm.invoke("张三25岁，并且有168厘米")
print(response)

response=llm.invoke("李四28岁，并且有172厘米")
print(response)

content='好的，根据您提供的信息，张三是25岁的男性，身高为168厘米。请问您接下来需要了解或讨论关于张三的哪些方面呢？如果是用于构建一个更详细的人物背景，或者如果有其他相关问题，请告诉我。' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T07:30:45.486678904Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 760356188, 'load_duration': 27512029, 'prompt_eval_count': 41, 'prompt_eval_duration': 49551000, 'eval_count': 54, 'eval_duration': 538008000} id='run-c8c1e4d9-c860-4cf6-bc60-7a7d934156a1-0' usage_metadata={'input_tokens': 41, 'output_tokens': 54, 'total_tokens': 95}
content='李四是位28岁的男士，身高172厘米。如果您需要更多关于他的信息，请提供更多细节或者具体问题。例如，您是否想知道他的体重、职业或者其他相关信息？' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-10-11T07:30:46.067307087Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 577732371, 'load_duration': 22191140, 'prompt_eval_count': 41, 'prompt_eval_duration': 27672000, 'eval_count': 40, 'ev

可以看出，直接问模型，没用定义结构化输出，模型输出内容不可控，下面对输出进行结构化处理

In [6]:
from typing import Optional
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(..., description="The height of the person expressed in meters.")

structured_llm = llm.with_structured_output(Person) # 可以绑定结构化输出
structured_llm.invoke("张三25岁，并且有168厘米")

Person(name='张三', height_in_meters=1.68)

可以看出，模型输出形成有固定结构的内容！
实际上langchain提供2种方式去设置LLMs的“结构化”输出，以下内容探索langchain结构化输出的原理

## 通过with_structured_output函数

In [3]:
import os
from langchain_ollama import ChatOllama
model_name = "qwen2.5:7b"
llm = ChatOllama(model=model_name)

from typing import Optional
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(..., description="The height of the person expressed in meters.")

structured_llm = llm.with_structured_output(Person) # 可以绑定结构化输出
structured_llm.invoke("张三25岁，并且有168厘米")

Person(name='张三', height_in_meters=1.68)

以上例子演示使用Pydantic class构建格式化的prompt,实际上with_structured_output接受以下输入：

- an OpenAI function/tool schema,
- a JSON Schema,
- a TypedDict class (support added in 0.1.20),
- or a Pydantic class.

## 自定义结构化输出

注意，并不是所有模型都实现了with_structured_output函数，因为并非所有模型都支持工具调用或 JSON 模式，此时有两种方法解决该问题：

- 使用PydanticOutputParser：利用内置类来解析与给定 Pydantic 模式匹配的聊天模型的输出
- 使用LCEL: 利用普通函数，自定义提示和解析器

## 使用PydanticOutputParser

In [4]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""
    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(..., description="The height of the person expressed in meters.")

class People(BaseModel):
    """Identifying information about all people in a text."""
    people: List[Person]

# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "张三25岁，并且有168厘米"
print(prompt.invoke(query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

可以看出这里prompt接受输入query，输出包含System和Human，其中System部分是按自定义结构输出生成的内容。

从System部分可以看出，为People类，生成了“As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.”，要求生成的json时，最好定义为{"foo": ["bar", "baz"]}而不是{"properties": {"foo": ["bar", "baz"]}}。

然后定义输出格式output schema，将这部分内容按json格式化后，显示如下，可以看出最外层要求输出people，而其被定义为Person的array，Person被要求输出name和height_in_meters。
```json
{
    "$defs": {
        "Person": {
            "description": "Information about a person.",
            "properties": {
                "name": {
                    "description": "The name of the person",
                    "title": "Name",
                    "type": "string"
                },
                "height_in_meters": {
                    "description": "The height of the person expressed in meters.",
                    "title": "Height In Meters",
                    "type": "number"
                }
            },
            "required": [
                "name",
                "height_in_meters"
            ],
            "title": "Person",
            "type": "object"
        }
    },
    "description": "Identifying information about all people in a text.",
    "properties": {
        "people": {
            "items": {
                "$ref": "#/$defs/Person"
            },
            "title": "People",
            "type": "array"
        }
    },
    "required": [
        "people"
    ]
}
```

这里就是通过修改输入LLMs的prompt，实现结构化内容输出，langchain的作用是实现“结构化表示->prompt”

In [5]:
chain = prompt | llm 

query = "张三25岁，并且有168厘米"
chain.invoke({"query": query})

'```json\n{\n  "people": [\n    {\n      "name": "张三",\n      "height_in_meters": 1.68\n    }\n  ]\n}\n```'

In [6]:
chain = prompt | llm | parser

query = "张三25岁，并且有168厘米"
chain.invoke({"query": query})

People(people=[Person(name='张三', height_in_meters=1.68)])

In [7]:
chain = prompt | llm | parser

query = "张三25岁，并且有168厘米并且李四28岁，并且有172厘米"
chain.invoke({"query": query}) # 输入包含多个实例也可以格式化输出

People(people=[Person(name='张三', height_in_meters=1.68), Person(name='李四', height_in_meters=1.72)])

总结：结构化输出，是通过修改prompt，让LLMs直接输出特定格式的内容。比如提问：“xxx，请以表格的形式显示”，而不是通过人工定义规则解析文本实现。关于如何实现“输出格式->prompt”，lanchain提供2种方式：

1. 使用with_structured_output：直接为操作直接绑定自定义结构化输出，自动生成prompt
2. 使用PydanticOutputParser：为操作自定义结构化输出，自动生成prompt

除此之外，还可以指定模型输出格式，并通过自定义函数解析替换以上的parser部分，实现结构化输出。综上可知，让模型输出输出结构化内容，langchain包括两个步骤：

1. 使用langchain定义结构化表示，并输出修改后的prompt到LLMs，LLMs输出结构化json的字符串格式内容
2. 解析LLMs输出为结构化内容

更多高级结构化参考 https://python.langchain.ac.cn/docs/how_to/structured_output/#choosing-between-multiple-schemas